In [7]:
import os
import torch as t
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.multiprocessing import Process
import transformers
import json
from io import StringIO

import sys
sys.path.append('/home/ubuntu/mlab/days')
import utils

DEVICES=[0,1]



def import_lesswrong_corpus():
    with open("/home/ubuntu/lw_corpus.json") as file:
        return json.load(file)
    return None

In [44]:
def corpus_to_tensor_list(corpus, max_tokens=1024):
    tokenizer = transformers.GPT2TokenizerFast.from_pretrained("gpt2")
    
    tensor_list = []
    current_tokens, staged_tokens = [], []
    
    for i, d in enumerate(corpus):
        print("Time for the: ", i)
        while True:
            if not len(staged_tokens) >= max_tokens:
                staged_tokens = tokenizer(d['title'] + d['text'] + '<|endoftext|>').input_ids
            while len(current_tokens) < max_tokens and len(staged_tokens) > max_tokens:

                space_in_current = max_tokens - len(current_tokens)            
                current_tokens += staged_tokens[0:space_in_current]
                staged_tokens = staged_tokens[space_in_current:None]
                print(len(staged_tokens), ":", staged_tokens)

            tensor_list.append(t.tensor(current_tokens))
            current_tokens = []
            break
        
    tensor_list.append(t.tensor(current_tokens))
    return tensor_list

In [20]:
import transformers
tokenizer = transformers.GPT2TokenizerFast.from_pretrained("gpt2")

In [45]:
corpus = import_lesswrong_corpus()
t_list = corpus_to_tensor_list(corpus, 10)
for tensor in t_list:
    tokens = tokenizer.convert_ids_to_tokens(tensor)
    print([token.replace('Ġ', '') for token in tokens])

Time for the:  0
28 : [2159, 10074, 9, 287, 262, 10470, 286, 383, 13248, 10933, 286, 3977, 22197, 329, 534, 3555, 329, 3707, 290, 9739, 13, 220, 29630, 36, 5959, 11, 12680, 50256]
Time for the:  1
18 : [286, 3977, 22197, 329, 534, 3555, 329, 3707, 290, 9739, 13, 220, 29630, 36, 5959, 11, 12680, 50256]
Time for the:  2
8 : [13, 220, 29630, 36, 5959, 11, 12680, 50256]
['title', '1', 'Project', 'Gutenberg', 'is', 'proud', 'to', 'cooperate', 'with', 'The']
['World', 'Library', '*', 'in', 'the', 'presentation', 'of', 'The', 'Complete', 'Works']
['of', 'William', 'Shakespeare', 'for', 'your', 'reading', 'for', 'education', 'and', 'entertainment']
[]
